### Extract only MFCC

In [1]:
import pandas as pd
#dev
import importlib as imp

import functions
imp.reload(functions)

from functions import *

import librosa

import os

import numpy as np

In [2]:
BASE_PATH = "datasets/DAIC-WOZ/"
files = os.listdir(BASE_PATH+"/cuts")
df_train = pd.read_csv(BASE_PATH + "train_split_Depression_AVEC2017.csv")
df_dev = pd.read_csv(BASE_PATH + "dev_split_Depression_AVEC2017.csv")
df_dev = df_dev[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score']]
df_train = df_train[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score']]

In [9]:
def process_segment(file_path, sr=16000):
    """
    Process each valid speech segment
    """

    audio, sr = librosa.load(file_path, sr=sr)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=sr,
        n_mfcc=40,  # 40 MFCC features
        hop_length=int(sr * 0.010),  # 10ms stride
        n_fft=int(sr * 0.025)  # 25ms window
    )
    
    # Normalize each feature
    mfcc = (mfcc - mfcc.mean(axis=1, keepdims=True)) / (mfcc.std(axis=1, keepdims=True) + 1e-10)
    
    return mfcc

def get_fixed_length_segments(mfcc, segment_length=200):
    """
    Ensure each MFCC has fixed length by either padding or truncating
    """
    current_length = mfcc.shape[1]
    
    if current_length > segment_length:
        # Take center part if too long
        start = (current_length - segment_length) // 2
        return mfcc[:, start:start+segment_length]
    elif current_length < segment_length:
        # Pad with zeros if too short
        padded = np.zeros((mfcc.shape[0], segment_length))
        start = (segment_length - current_length) // 2
        padded[:, start:start+current_length] = mfcc
        return padded
    else:
        return mfcc

In [14]:
# Process all files
base_path = BASE_PATH + "cuts"
all_mfccs = []
all_labels = []

for _, row in df_dev.iterrows():
    patient = row['Participant_ID']
    print(patient)
    label = row['PHQ8_Binary']  # Assuming this is your label
    
    patient_files = [f for f in os.listdir(base_path) 
                    if f.startswith(str(patient)) and f.endswith('.wav')]
    
    for f in patient_files:
        file_path = f"{base_path}/{f}"
        try:
            # Extract MFCC
            mfcc = process_segment(file_path)
            
            # Ensure fixed length
            mfcc = get_fixed_length_segments(mfcc, segment_length=200)
            
            # Store MFCC and label
            all_mfccs.append(mfcc)
            all_labels.append(label)
            
            # Print shapes occasionally to verify
            if len(all_mfccs) % 100 == 0:
                print(f"Processed {len(all_mfccs)} segments. Current MFCC shape: {mfcc.shape}")
                
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue

# Convert to numpy arrays
X = np.array(all_mfccs)
y = np.array(all_labels)

# save X and y into a .npz file
np.savez_compressed('DAIC_test.npz', X=X, y=y)

print(f"Final dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")

302
307
Processed 100 segments. Current MFCC shape: (40, 200)
331
335
Processed 200 segments. Current MFCC shape: (40, 200)
346
Processed 300 segments. Current MFCC shape: (40, 200)
367
377
Processed 400 segments. Current MFCC shape: (40, 200)
381
382
Processed 500 segments. Current MFCC shape: (40, 200)
388
389
Processed 600 segments. Current MFCC shape: (40, 200)
390
Processed 700 segments. Current MFCC shape: (40, 200)
395
403
Processed 800 segments. Current MFCC shape: (40, 200)
404
406
Processed 900 segments. Current MFCC shape: (40, 200)
413
417
Processed 1000 segments. Current MFCC shape: (40, 200)
418
420
Processed 1100 segments. Current MFCC shape: (40, 200)
422
436
Processed 1200 segments. Current MFCC shape: (40, 200)
439
440
Processed 1300 segments. Current MFCC shape: (40, 200)
451
458
472
Processed 1400 segments. Current MFCC shape: (40, 200)
476
477
Processed 1500 segments. Current MFCC shape: (40, 200)
482
483
Processed 1600 segments. Current MFCC shape: (40, 200)
484
4

In [ ]:
np.savez_compressed('DAIC_train.npz', X=X, y=y)

In [ ]:
base_path = BASE_PATH+"cuts"

for _, row in df_train.iterrows():
    patient = row['Participant_ID']
    patient_files = [f for f in os.listdir(base_path) if f.startswith(str(patient)) and f.endswith('.wav')]
    print(patient_files)s
    for f in patient_files:
        file_path = f"{base_path}/{f}"
        print(file_path)
        mfcc = process_segment(file_path)
        print(mfcc)


['303_segment_0.wav', '303_segment_1.wav', '303_segment_10.wav', '303_segment_11.wav', '303_segment_12.wav', '303_segment_13.wav', '303_segment_14.wav', '303_segment_15.wav', '303_segment_16.wav', '303_segment_17.wav', '303_segment_18.wav', '303_segment_19.wav', '303_segment_2.wav', '303_segment_20.wav', '303_segment_21.wav', '303_segment_22.wav', '303_segment_23.wav', '303_segment_24.wav', '303_segment_25.wav', '303_segment_26.wav', '303_segment_27.wav', '303_segment_28.wav', '303_segment_29.wav', '303_segment_3.wav', '303_segment_30.wav', '303_segment_31.wav', '303_segment_32.wav', '303_segment_33.wav', '303_segment_34.wav', '303_segment_35.wav', '303_segment_36.wav', '303_segment_37.wav', '303_segment_38.wav', '303_segment_39.wav', '303_segment_4.wav', '303_segment_40.wav', '303_segment_41.wav', '303_segment_42.wav', '303_segment_43.wav', '303_segment_44.wav', '303_segment_45.wav', '303_segment_46.wav', '303_segment_47.wav', '303_segment_48.wav', '303_segment_49.wav', '303_segment_5

ParameterError: Audio data must be of type numpy.ndarray